# Sign In, Load Data

In [1]:
# 0. Cleaning Data
# 
# download the survey's .csv file from limesurvey
# remember to always use the .csv file with all the columns (select all columns)
# completed answers ('apenas respostas completas')
# answers' code ('codigos das respostas')
# question's code and complete question ('codigo e texto da pergunta')
#rename this file to data_raw_#.csv , being # the number of answers

#also export the complete answers of the nationalities and rename it to countries_answers_NUMBEROFANSWERS 
# ('respostas completas'; 'texto completo da pergunta' da pergunta/coluna NATIONALITY)

import pandas as pd
df = pd.read_csv('data_raw_188.csv')
df2 = pd.read_csv('countries_answers_188.csv')

In [2]:
#a ckeck to see if it's actually the correct number of answers
num_observations = len(df)

print(num_observations)

188


#  _Well Defined_: No Adjustments Needed After New Raw Dataset 

## Transforming *Questions* into Clean Variables

there are three main groups of questions: 1. preferences; 2. socio-demographics ; and 3. travel behavior

In [3]:
#dropping the unnecessary columns
df = df.drop(['lastpage. Last page', 'seed. Seed', 'startlanguage. Start language','submitdate. Date submitted'], axis=1)

#renaming some questions to clean variable names (e.g., socio-demographics questions)
df = df.rename(columns={'id. Response ID' : 'id'})
df = df.rename(columns={'randGroup. {if(randGroup &gt; 0, randGroup , rand(1,4))}  \xa0 ' : 'group'})
df = df.rename(columns={'CAR. Do you own and have access to a private car?' : 'own_car'})
df = df.rename(columns={'PARKING[SQ001]. Do you have access to free parking or do you have a parking permit for your residential area and your workplace/university? [At Home Area]' : 'parking_home'})
df = df.rename(columns={'PARKING[SQ002]. Do you have access to free parking or do you have a parking permit for your residential area and your workplace/university? [At Workplace/University]' : 'parking_work'})
df = df.rename(columns={'COMMUTETIME. How long does your daily commute approximately take? ' : 'commute_time'})
df = df.rename(columns={'AGE. What is your age?\xa0' : 'age'})
df = df.rename(columns={'GENDER. What is your gender identity?' : 'gender'})
df = df.rename(columns={'OCCUPATION. What is your current occupation?' : 'occupation'})
df = df.rename(columns={'RESIDENCE[SQ001]. Where do you live? [City/Town]' : 'residence_city'})
df = df.rename(columns={'RESIDENCE[SQ002]. Where do you live? [Country]' : 'residence_country'})
df = df.rename(columns={'NATIONALITY. What is your country of origin?' : 'nationality'})
df = df.rename(columns={'CHILDREN. Do you have children?' : 'children'})
df = df.rename(columns={'SPECIALIST. Do you have any background, either past or present, through study or work, in the fields of urban mobility, transportation, or urbanism?':'specialist'})

In [4]:
#renaming the columns names/variables of the preferences group (preference;context;likert scales)

def generate_new_column_names(existing_columns):
    new_column_names = {}
    num_columns = len(existing_columns)
    batch_size = 10
    max_batches = 8  # Number of prefixes per hundred series (e.g., G101 to G108)
    suffixes = [
        '_preference', '_context', '_usefulness', '_safety', '_security', 
        '_stress', '_comfort', '_order', '_liveability', '_more'
    ]
    
    # Ensure 'group' column is not renamed
    new_column_names['group'] = 'group'  # The first column remains "group"
    
    # Start renaming from the second column
    start_index = 2  # The first index after 'group'
    
    for hundred_prefix in range(1, 5):  # G101 to G408 (4 hundred series)
        for batch_index in range(max_batches):
            prefix = f'G{hundred_prefix * 100 + batch_index + 1}'
            end_index = start_index + batch_size
            
            if start_index >= num_columns:
                break

            for i in range(start_index, min(end_index, num_columns)):
                old_name = existing_columns[i]
                suffix_index = i - start_index
                new_name = f'{prefix}{suffixes[suffix_index]}'
                new_column_names[old_name] = new_name

            start_index = end_index

    return new_column_names


# Generate new column names based on the existing DataFrame columns
existing_columns = df.columns
new_column_names = generate_new_column_names(existing_columns)

# Rename the columns
df.rename(columns=new_column_names, inplace=True)

In [5]:
# renaming the columns names/variables of the socio-demographics group (children, pets, disabilities), travel behavior group (Commute), and specialists background (roles)

# Define a mapping function based on column patterns
def generate_new_column_name(old_name):
    if 'CHILDRENNO' in old_name:
        age_group = old_name.split('[')[-1].split(']')[0]
        return f'children_{age_group}'
    elif 'PETS' in old_name:
        pet_type = old_name.split('[')[-1].split(']')[0]
        return f'pet_{pet_type}'
    elif 'DISABILITIES' in old_name:
        disability_type = old_name.split('[')[-1].split(']')[0]
        return f'disability_{disability_type}'
    elif 'SPECIALISTTYPE' in old_name:
        role_type = old_name.split('[')[-1].split(']')[0]
        return f'role_{role_type}'
    elif 'COMMUTE' in old_name:
        mode_of_transport = old_name.split('[')[-1].split(']')[0]
        return f'commute_{mode_of_transport}'
    else:
        return old_name  # Return the same name if no match

# Create a mapping dictionary
new_column_names = {col: generate_new_column_name(col) for col in df.columns}

# Rename the columns
df.rename(columns=new_column_names, inplace=True)

In [6]:
#another adjustment to have the clean variables' names 
df = df.rename(columns={'commute_I always work from home':'commute_home'})
df = df.rename(columns={'children_0 - 5 years old':'children_0-5'})
df = df.rename(columns={'children_6 - 10 years old':'children_6-10'})
df = df.rename(columns={'children_11 years old and above':'children_11'})
df = df.rename(columns={'pet_Yes, I have one (or more) dog(s)':'pet_Y_dog'})
df = df.rename(columns={'pet_Yes, I have another type of pet':'pet_Y_other'})
df = df.rename(columns={'pet_No, I do not have pets':'pet_N'})
df = df.rename(columns={'disability_Yes, I have reduced mobility':'disability_Y_mobility'})
df = df.rename(columns={'disability_Yes, I have a visual impairment':'disability_Y_visual'})
df = df.rename(columns={'disability_Yes, I have a hearing impairment':'disability_Y_hearing'})
df = df.rename(columns={'disability_Yes, I have another type of disability':'disability_Y_other'})
df = df.rename(columns={'disability_No, I do not have any disabilities':'disability_N'})
df = df.rename(columns={'commute_Taxi/TVDE (e.g. Uber, Bolt)': 'commute_TVDE'})
df = df.rename(columns={'role_Academic/Researcher': 'role_academic'})
df = df.rename(columns={'role_Professional (e.g., Planner, Engineer)': 'role_professional'})
df = df.rename(columns={'role_Government/Public Sector (e.g., city planner, policy maker, transit operator)': 'role_public'})
df = df.rename(columns={'role_Industry/Private Sector (e.g., consultant, analyst, developer)': 'role_private'})
df = df.rename(columns={'role_ Non-Profit/Advocacy': 'role_nonprofit'})
df = df.rename(columns={'role_Technology/Innovation (e.g., developer, data analyst)': 'role_technology'})
df = df.rename(columns={'role_Other': 'role_other'})

## Transforming *Answers* into Clean Values

### Renaming Values of Preference for All Groups of Interventions

The variables were renamed the following way:
 _original value from Limesurvey_ (AO01 or AO02)  →  *X_Y_Z_W* , where:

1. X = Intervention_ID (check annexed table, named _FALTAAQUI_)
2. Y = Type of Intervention (check annexed table, named _FALTAAQUI_)
3. Z = Scale/Dimension of Intervention (l for large; m for medium; t for transformative)
4. W = Before Intervention (b) or After Intervention (a)

note: the values of A001 and A002 correspond to before or after the intervention, depending on how I've positioned the images in LimeSurvey. To avoid influencing participants’ decisions, the images representing "before" and "after" for each intervention were placed randomly on either the left or right side.

In [7]:
#group 1
df['G101_preference'] = df['G101_preference'].replace('AO01', '2_pedestrian_l_a')
df['G101_preference'] = df['G101_preference'].replace('AO02', '2_pedestrian_l_b')

df['G102_preference'] = df['G102_preference'].replace('AO01', '7_shared_m_b')
df['G102_preference'] = df['G102_preference'].replace('AO02', '7_shared_m_a')

df['G103_preference'] = df['G103_preference'].replace('AO01', '55_pedestrian_l_a')
df['G103_preference'] = df['G103_preference'].replace('AO02', '55_pedestrian_l_b')

df['G104_preference'] = df['G104_preference'].replace('AO01', '18_transit_m_a')
df['G104_preference'] = df['G104_preference'].replace('AO02', '18_transit_m_b')

df['G105_preference'] = df['G105_preference'].replace('AO01', '49_free_t_b')
df['G105_preference'] = df['G105_preference'].replace('AO02', '49_free_t_a')

df['G106_preference'] = df['G106_preference'].replace('AO01', '37_car_t_a')
df['G106_preference'] = df['G106_preference'].replace('AO02', '37_car_t_b')

df['G107_preference'] = df['G107_preference'].replace('AO01', '33_transit_l_a')
df['G107_preference'] = df['G107_preference'].replace('AO02', '33_transit_l_b')

df['G108_preference'] = df['G108_preference'].replace('AO02', '30_car_m_a')
df['G108_preference'] = df['G108_preference'].replace('AO01', '30_car_m_b')


In [8]:
#group 2
df['G201_preference'] = df['G201_preference'].replace('AO01', '63_free_t_a')
df['G201_preference'] = df['G201_preference'].replace('AO02', '63_free_t_b')

df['G202_preference'] = df['G202_preference'].replace('AO01', '60_shared_l_b')
df['G202_preference'] = df['G202_preference'].replace('AO02', '60_shared_l_a')

df['G203_preference'] = df['G203_preference'].replace('AO01', '37_car_t_a')
df['G203_preference'] = df['G203_preference'].replace('AO02', '37_car_t_b')

df['G204_preference'] = df['G204_preference'].replace('AO01', '34_car_m_b')
df['G204_preference'] = df['G204_preference'].replace('AO02', '34_car_m_a')

df['G205_preference'] = df['G205_preference'].replace('AO01', '72_transit_m_b')
df['G205_preference'] = df['G205_preference'].replace('AO02', '72_transit_m_a')

df['G206_preference'] = df['G206_preference'].replace('AO01', '25_transit_l_a')
df['G206_preference'] = df['G206_preference'].replace('AO02', '25_transit_l_b')

df['G207_preference'] = df['G207_preference'].replace('AO01', '4_pedestrian_l_b')
df['G207_preference'] = df['G207_preference'].replace('AO02', '4_pedestrian_l_a')

df['G208_preference'] = df['G208_preference'].replace('AO01', '15_pedestrian_m_b')
df['G208_preference'] = df['G208_preference'].replace('AO02', '15_pedestrian_m_a')

In [9]:
#grupo 3
df['G301_preference'] = df['G301_preference'].replace('AO01', '64_car_l_b')
df['G301_preference'] = df['G301_preference'].replace('AO02', '64_car_l_a')

df['G302_preference'] = df['G302_preference'].replace('AO01', '9_pedestrian_m_b')
df['G302_preference'] = df['G302_preference'].replace('AO02', '9_pedestrian_m_a')

df['G303_preference'] = df['G303_preference'].replace('AO01', '50_pedestrian_l_b')
df['G303_preference'] = df['G303_preference'].replace('AO02', '50_pedestrian_l_a')

df['G304_preference'] = df['G304_preference'].replace('AO01', '31_car_m_a')
df['G304_preference'] = df['G304_preference'].replace('AO02', '31_car_m_b')

df['G305_preference'] = df['G305_preference'].replace('AO01', '69_transit_m_a')
df['G305_preference'] = df['G305_preference'].replace('AO02', '69_transit_m_b')

df['G306_preference'] = df['G306_preference'].replace('AO01', '66_transit_l_a')
df['G306_preference'] = df['G306_preference'].replace('AO02', '66_transit_l_b')

df['G307_preference'] = df['G307_preference'].replace('AO01', '39_pedestrian_t_a')
df['G307_preference'] = df['G307_preference'].replace('AO02', '39_pedestrian_t_b')

df['G308_preference'] = df['G308_preference'].replace('AO01', '67_shared_t_b')
df['G308_preference'] = df['G308_preference'].replace('AO02', '67_shared_t_a')

In [10]:
#grupo 4
df['G401_preference'] = df['G401_preference'].replace('AO01', '18_transit_m_a')
df['G401_preference'] = df['G401_preference'].replace('AO02', '18_transit_m_b')

df['G402_preference'] = df['G402_preference'].replace('AO01', '41_shared_t_a')
df['G402_preference'] = df['G402_preference'].replace('AO02', '41_shared_t_b')

df['G403_preference'] = df['G403_preference'].replace('AO01', '20_pedestrian_l_b')
df['G403_preference'] = df['G403_preference'].replace('AO02', '20_pedestrian_l_a')

df['G404_preference'] = df['G404_preference'].replace('AO01', '10_pedestrian_l_b')
df['G404_preference'] = df['G404_preference'].replace('AO02', '10_pedestrian_l_a')


df['G405_preference'] = df['G405_preference'].replace('AO01', '64_car_l_b')
df['G405_preference'] = df['G405_preference'].replace('AO02', '64_car_l_a')

df['G406_preference'] = df['G406_preference'].replace('AO01', '69_transit_m_a')
df['G406_preference'] = df['G406_preference'].replace('AO02', '69_transit_m_b')

df['G407_preference'] = df['G407_preference'].replace('AO01', '62_free_t_a')
df['G407_preference'] = df['G407_preference'].replace('AO02', '62_free_t_b')

df['G408_preference'] = df['G408_preference'].replace('AO01', '68_car_m_a')
df['G408_preference'] = df['G408_preference'].replace('AO02', '68_car_m_b')

### Renaming Values of Influential Factors on Preference

5 Point Likert Scale (with the original values of Limesurvey) to -2 to 2 (extreme negative influence, negative influence, no influence, positive influence, extreme positive influence)

In [11]:
#im going to change the likert scale now from AO05 - 5 
# Mapping dictionary
mapping = {
    'AO01': -2,
    'AO02': -1,
    'AO03': 0,
    'AO04': 1,
    'AO05': 2
}

# Define the prefixes and the suffixes you want to process
prefixes = ['G101', 'G102', 'G103', 'G104', 'G105', 'G106', 'G107', 'G108',
            'G201', 'G202', 'G203', 'G204', 'G205', 'G206', 'G207', 'G208',
            'G301', 'G302', 'G303', 'G304', 'G305', 'G306', 'G307', 'G308',
            'G401', 'G402', 'G403', 'G404', 'G405', 'G406', 'G407', 'G408']
suffixes = ['usefulness', 'safety', 'security', 'stress', 'comfort', 'order', 'liveability']

# Generate the list of columns to replace
columns_to_replace = [f"{prefix}_{suffix}" for prefix in prefixes for suffix in suffixes if f"{prefix}_{suffix}" in df.columns]

# Replace values using the mapping
df[columns_to_replace] = df[columns_to_replace].replace(mapping)

C:\Users\maryi\AppData\Local\Temp\ipykernel_22800\2930454039.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[columns_to_replace] = df[columns_to_replace].replace(mapping)


### Renaming Socio-Demographics Answers to Clean Values

In [12]:
df['gender'] = df['gender'].replace('AO01', 'F')
df['gender'] = df['gender'].replace('AO02', 'M')
df['gender'] = df['gender'].replace('AO03', 'N')
df['gender'] = df['gender'].replace('AO04', 'O')


df['commute_time'] = df['commute_time'].replace('AO01', '<15')
df['commute_time'] = df['commute_time'].replace('AO02', '15-30')
df['commute_time'] = df['commute_time'].replace('AO03', '30-45')
df['commute_time'] = df['commute_time'].replace('AO05', '+45')
df['commute_time'] = df['commute_time'].replace('AO06', 'home')
#home = 0 in the scale


df['parking_home'] = df['parking_home'].replace('AO01', 'free')
df['parking_home'] = df['parking_home'].replace('AO02', 'permit')
df['parking_home'] = df['parking_home'].replace('AO03', 'not free')

df['parking_work'] = df['parking_work'].replace('AO01', 'free')
df['parking_work'] = df['parking_work'].replace('AO02', 'permit')
df['parking_work'] = df['parking_work'].replace('AO03', 'not free')

df['occupation'] = df['occupation'].replace('SQ002', 'Employee')
df['occupation'] = df['occupation'].replace('SQ003', 'Retired')
df['occupation'] = df['occupation'].replace('SQ004', 'Unemployed')
df['occupation'] = df['occupation'].replace('SQ005', 'Undergrad student')
df['occupation'] = df['occupation'].replace('SQ006', 'Graduate student')
df['occupation'] = df['occupation'].replace('SQ007', 'Freelancer')
df['occupation'] = df['occupation'].replace('SQ008', 'Business owner')
df['occupation'] = df['occupation'].replace('SQ009', 'Researcher')
df['occupation'] = df['occupation'].replace('SQ010', 'Working Student')

df['age'] = df['age'].replace('54 anos ', '54')


# Define a dictionary for the replacements
replacements = {
    'AO01': '1',
    'AO02': '2',
    'AO03': '3'  # or more
}

df[['children_0-5', 'children_6-10', 'children_11']] = (
    df[['children_0-5', 'children_6-10', 'children_11']]
    .replace(replacements)
    .fillna(0)  # Replace NaN with 0
    .astype(int)  # Convert to integer type
)

# _Final Clean_: Some Adjustments May Be Required After New Raw Dataset

### Cleaning Residency and Nationality

In [13]:
#renaming variable nationality
df['nationality'] = df2['What is your country of origin?'] 

In [14]:
#cleaning variables, that were open-ended 

# removing the leading or trailing spaces of the open-ended answers of residence city and country
df['residence_city'] = df['residence_city'].str.strip()
df['residence_country'] = df['residence_country'].str.strip()

#normalization of all text inputs! (residence_city and residence_country), removing accents and putting everything in lowercase
from unidecode import unidecode

df['residence_city'] = df['residence_city'].apply(lambda x: unidecode(x.lower().strip()))
df['residence_country'] = df['residence_country'].apply(lambda x: unidecode(x.lower().strip()))
df['nationality'] = df['nationality'].apply(lambda x: unidecode(x.lower().strip()))

In [15]:
# check if there are any redundancies on the cities or countries (open-ended), particularly to check if there's anything written in a different way or a different language
unique_cities = df['residence_city'].unique()
unique_countries = df['residence_country'].unique()

print(f"Unique values in column 'residence_city': {unique_cities}")
print(f"Unique values in column 'residence_country': {unique_countries}")

Unique values in column 'residence_city': ['lisbon' 'cascais' 'luanda' 'sintra' 'covilha' 'sao paulo' 'coimbra'
 'almada' 'oeiras' 'natal/rn' 'natal' 'landegem' 'barcelona' 'amersfoort'
 'porto' 'corroios' 'basel' 'manaus' 'rio de janeiro' 'guarda' 'stockholm'
 'belo horizonte' 'brussels' 'madrid' 'budapest' 'lisboa' 'ghent'
 'helsinki' 'aveiro' 'gothenburg' 'athens' 'munich' 'dorchester' 'arganil'
 'seixal' 'braga' 'city' 'manaus-am' 'copenhagen' 'halifax' 'sao carlos'
 'parede' 'enschede' 'santiago' 'munster' 'mafra' 'bremen' 'mainz'
 'wiesbaden' 'freiburg' 'hanover' 'essen' 'lorrach' 'frankfurt am main'
 'milano' 'haifa' 'kuwait' 'cologne' 'hamburg' '2' 'villars-sur-glane'
 'setubal' 'stuttgart' 'tokyo' 'dublin' 'atlanta' 'london' 'recife'
 'utrecht' 'tartu' 'madalena' 'antwerp' 'cascais - lisbon' 'florianopolis'
 'berlin' 'fortaleza/ceara']
Unique values in column 'residence_country': ['portugal' 'angola' 'brasil' 'brazil' 'belgium' 'spain' 'the netherlands'
 'switzerland' 'sweden'

In [16]:
#alteração de todas as cidades e países 
# Create a mapping dictionary for normalization
city_mapping = {'lisboa': 'lisbon', 'natal/rn':'natal','manaus-am': 'manaus','frankfurt am main': 'frankfurt','cascais - lisbon': 'cascais','city': 'other','2': 'other'}
country_mapping = {'brasil':'brazil','the netherlands':'netherlands','potugal':'portugal','country': 'other','2':'other','alemanha':'germany', 'schweiz':'switzerland'}

# Apply the mapping to normalize the city and country names
df['residence_city'] = df['residence_city'].replace(city_mapping)
df['residence_country'] = df['residence_country'].replace(country_mapping)

### Cleaning Other Variables and Its Values

In [17]:
# Iterate over each column and print the unique values

for column in df.columns[-36:]:
    unique_values = df[column].unique()
    print(f"Unique values in column '{column}': {unique_values}")

Unique values in column 'children': ['N' 'Y']
Unique values in column 'children_0-5': [0 1 2]
Unique values in column 'children_6-10': [0 1 2]
Unique values in column 'children_11': [0 2 1 3]
Unique values in column 'pet_Y_dog': [nan 'Y']
Unique values in column 'pet_Y_other': [nan 'Y']
Unique values in column 'pet_N': ['Y' nan]
Unique values in column 'disability_Y_mobility': [nan 'Y']
Unique values in column 'disability_Y_visual': [nan]
Unique values in column 'disability_Y_hearing': [nan]
Unique values in column 'disability_Y_other': [nan 'Y']
Unique values in column 'disability_N': ['Y' nan]
Unique values in column 'occupation': ['Graduate student' 'Employee' 'Unemployed' 'Working Student'
 'Business owner' 'Researcher' 'Freelancer' 'Retired' 'Undergrad student']
Unique values in column 'specialist': ['N' 'Y']
Unique values in column 'role_academic': [nan 'Y']
Unique values in column 'role_professional': [nan 'Y']
Unique values in column 'role_public': [nan 'Y']
Unique values in co

In [18]:
# Find unique values in the 'commute_other' column
unique_disability_visual = df['disability_Y_visual'].unique()
unique_disability_hearing = df['disability_Y_hearing'].unique()

# Print unique values
print(unique_disability_visual)
print(unique_disability_hearing)

[nan]
[nan]


In [19]:
#Until the moment (188 answers), there are no answers to some of the columns, so i deleted those
# columns that dont have information and also remove the "more" since they have phrases with commas, and no relevant input was done 
df = df.drop(['disability_Y_visual','disability_Y_hearing'], axis=1)

df = df.drop(df.filter(regex='_more$').columns, axis=1)

In [20]:
# Find unique values in the 'commute_other' column
unique_commute_others = df['commute_Other'].unique()

# Print unique values
print(unique_commute_others)

[nan 'Bus' 'tram' 'Boat' 'Tramway' 's(peed)-Pedelec'
 'I work from home most of the time, but when I commute to the office I use the train, subway, TVDE and I walk on foot. Occasionally I use the bicycle or the bus.'
 'Carsharing']


# Exporting Clean File to Further Encode It

In [22]:
file_path = 'C:\\Users\\maryi\\Desktop\\Tese Data Analysis\\data_clean.csv'

# Export the DataFrame to a .csv file
df.to_csv(file_path, index=False)